# What is about 

That script gives example how to load multiple genes expression datasets on kaggle.
(Some other datasets like Dexter included for comparaison). 
There are several of them available on kaggle, we collect them.  

Horizontal axis - genes, Vertical axis - cells/samples. 


V6 - added:
    1) Melanoma Gene expressions https://www.kaggle.com/jomaminoza/melanoma-gene-expressions
    473 rows × 20530 columns. 
    Seems to be scRNA seq data, refrences are absent. 
    2) Mice Protein Expression  https://www.kaggle.com/ruslankl/mice-protein-expression
    Classical dataset even mentioned in sklearn as example for access to openml datasets.
    Expression level of 77 proteins are measured after for 72 mices (and many measurements for each) - in total 1080 measurements. The data set consists of the expression levels of 77 proteins/protein modifications that produced detectable signals in the nuclear fraction of cortex. There are 38 control mice and 34 trisomic mice (Down syndrome), for a total of 72 mice. In the experiments, 15 measurements were registered of each protein per sample/mouse. Therefore, for control mice, there are 38x15, or 570 measurements, and for trisomic mice, there are 34x15, or 510 measurements. The dataset contains a total of 1080 measurements per protein.
    Dataset at UC Irvine: https://archive.ics.uci.edu/ml/datasets/Mice+Protein+Expression
    There is nice notebook for the dataset: https://www.kaggle.com/ruslankl/protein-expression-analysis
    
V5 - added:
    Breast Cancer Proteomes - https://www.kaggle.com/piotrgrabo/breastcancerproteomes
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6547580/ https://www.nature.com/articles/s41598-019-44339-4.pdf?origin=ppub
Data - log2 iTRAQ ratios for each sample (protein expression data, most important), three last are from healthy individuals. RefSeq_accession_number: RefSeq protein ID (each protein has a unique ID in a RefSeq database) gene_symbol: a symbol unique to each gene (every protein is encoded by some gene) gene_name: a full name of that gene Remaining columns.
83 rows × 12553 columns 


V4 - added:
    1) Brain cancer microarray GSE50161 from CuMiDa: https://www.kaggle.com/brunogrisci/brain-cancer-gene-expression-cumida
        5 classes 54676 genes 130 samples .  4 different types of brain cancer (plus healthy tissue)
    2) Breast cancer GSE45827 microarray from CuMiDa https://www.kaggle.com/brunogrisci/breast-cancer-gene-expression-cumida
        6 classes, 54676 genes, 151 samples

V3 - added:
    Leukemia microarray gene expression GSE9476 from CuMiDa:  https://www.kaggle.com/brunogrisci/leukemia-gene-expression-cumida
    5 classes, 22284 genes, 64 samples


V2 - added:
    Dataset by Golub et.al. 1999 - see info:  https://www.kaggle.com/crawford/gene-expression
    It showed how new cases of cancer could be classified by gene expression monitoring (via DNA microarray) 
    These data were used to classify patients with acute myeloid leukemia (AML) and acute lymphoblastic leukemia (ALL).
    shape: 72 x 7219 

V1 - 6 datasets: 

OVA_Prostate dataset from openml 

single-cell-rna-seq-from-stoeckius-et-al-2017/GSE100866_CD8_merged-RNA_umi

GSE67123_v6_scrna_10x_143_23548_Mouse_Embryo_HSCs_invivo_fromCytotrace.csv

human_liver from ARCHS4

GSE90047_Mouse_Liver_Hepatoblast_invivo

Dexter300 dataset 


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.neighbors import kneighbors_graph
import time
import matplotlib.pyplot as plt
import seaborn as sns

try:
    import cudf
    from cuml.neighbors import NearestNeighbors
    #from cuml.datasets import make_blobs    
except:
    print('GPU library cannot be imported. Turn ON GPU')
    

# Load Data

In [ ]:
pip install openml


In [ ]:
import openml

t0 = time.time()
openml_df = openml.datasets.list_datasets(output_format="dataframe")
print(time.time()-t0,'seconds passed ')

if 0: ## The same can be done with BIGGER lines of code
    openml_list = openml.datasets.list_datasets()  # returns a dict
    # Show a nice table with some key data properties
    openml_df = pd.DataFrame.from_dict(openml_list, orient="index")
    openml_df = datalist[["did", "name", "NumberOfInstances", "NumberOfFeatures", "NumberOfClasses"]]

    print(f"First 10 of {len(datalist)} datasets...")
    openml_df.head(n=10)


openml_df

# Among all datasets there is subset of 45 datasets with  mircoarray Gene Expressions (mainly cancer) 
# can be selected by number of features == 10937
df_gemler = openml_df[ ( openml_df.NumberOfFeatures == 10937)    ].sort_values(["NumberOfInstances"], ascending=False)#.head(n=20)
print(df_gemler.shape)
df_gemler

In [ ]:

def get_Golub_1999_gen_expressions_for_leukemias():
    '''
    Load dataset by Golub et.al. 1999 - see info:  https://www.kaggle.com/crawford/gene-expression
    This dataset comes from a proof-of-concept study published in 1999 by Golub et al. 
    It showed how new cases of cancer could be classified by gene expression monitoring (via DNA microarray) 
    and thereby provided a general approach for identifying new cancer classes and assigning tumors to known classes. 
    These data were used to classify patients with acute myeloid leukemia (AML) and acute lymphoblastic leukemia (ALL).
    
    We join train and test, and throw out some "Gene Description", "Gene Accession Number"
    to get standard form of datasets. 
    
    There are two datasets containing the initial (training, 38 samples) and independent (test, 34 samples) datasets used in the paper. These datasets contain measurements corresponding to ALL and AML samples from Bone Marrow and Peripheral Blood. Intensity values have been re-scaled such that overall intensities for each chip are equivalent.
    '''
    
    f_test = '/kaggle/input/gene-expression/data_set_ALL_AML_independent.csv'
    f_train = '/kaggle/input/gene-expression/data_set_ALL_AML_train.csv'
    f_y = '/kaggle/input/gene-expression/actual.csv'    
    
    y = pd.read_csv(f_y)# '../input/actual.csv') 
    y = y.replace({'ALL':0,'AML':1})
    y = y.set_index('patient')
    y = y['cancer']

    
    # Import training data
    df_train = pd.read_csv(f_train)#'../input/data_set_ALL_AML_train.csv')
    #print(df_train.shape)

    # Import testing data
    df_test = pd.read_csv(f_test)# '../input/data_set_ALL_AML_independent.csv')
    #print(df_test.shape)

    train_to_keep = [col for col in df_train.columns if "call" not in col]
    test_to_keep = [col for col in df_test.columns if "call" not in col]

    X_train_tr = df_train[train_to_keep]
    X_test_tr = df_test[test_to_keep]

    train_columns_titles = ['Gene Description', 'Gene Accession Number', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
           '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', 
           '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38']

    X_train_tr = X_train_tr.reindex(columns=train_columns_titles)

    test_columns_titles = ['Gene Description', 'Gene Accession Number','39', '40', '41', '42', '43', '44', '45', '46',
           '47', '48', '49', '50', '51', '52', '53',  '54', '55', '56', '57', '58', '59',
           '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72']

    X_test_tr = X_test_tr.reindex(columns=test_columns_titles)

    X_train = X_train_tr.T
    X_test = X_test_tr.T

    X = pd.concat( [X_train, X_test], axis = 0 )
    X
    X.columns = X.iloc[1]
    X = X.drop(["Gene Description", "Gene Accession Number"]).apply(pd.to_numeric)
    X    

    return X,y


In [ ]:
dataset = 'human_liver_ARCHS4' # '' 
dataset = 'Golub_1999_gen_expressions_for_leukemias'
dataset = 'Leukemia GSE9476 microarray CuMiDa'
dataset = 'Breast cancer GSE45827 microarray CuMiDa'
dataset = 'Brain cancer GSE50161 microarray CuMiDa'
dataset = 'Breast Cancer Proteomes'
dataset = 'Melanoma RNAseq'

dataset = 'Mice Protein Expression'


if dataset == 'Mice Protein Expression':
    fn = '/kaggle/input/mice-protein-expression/Data_Cortex_Nuclear.csv'
    df = pd.read_csv(fn, index_col = 0)
    y = df['class']
    X = df.iloc[:,:-4]
    
if dataset == 'Melanoma RNAseq':
    fn = '/kaggle/input/melanoma-gene-expressions/MDB_SKCM_RNASeq to joma.csv'
    df = pd.read_csv(fn, index_col = 0)
    X = df.T

if dataset == 'Breast Cancer Proteomes':
    fn = '/kaggle/input/breastcancerproteomes/77_cancer_proteomes_CPTAC_itraq.csv'
    df = pd.read_csv(fn)# , index_col = 0)
    t = df['RefSeq_accession_number'] + df['gene_symbol'].apply(lambda x: ' ' + str (x) + ' ' ) + df['gene_name']
    df.index = t 
    df = df.iloc[:,3:]
    df.index.name = 'Genes Info'
    X = df.T
    y = pd.Series(index = X.index, data = np.zeros(len(X)))
    y.iloc[-3:] = 1 # 3 healthy 

if dataset == 'Brain cancer GSE50161 microarray CuMiDa':
    fn = '/kaggle/input/brain-cancer-gene-expression-cumida/Brain_GSE50161.csv'
    df = pd.read_csv(fn, index_col = 0)
    y = df['type']
    X = df.iloc[:,1:]


if dataset == 'Breast cancer GSE45827 microarray CuMiDa':
    fn = '/kaggle/input/breast-cancer-gene-expression-cumida/Breast_GSE45827.csv'
    df = pd.read_csv(fn, index_col = 0)
    y = df['type']
    X = df.iloc[:,1:]

if dataset == 'Leukemia GSE9476 microarray CuMiDa':
    fn = '/kaggle/input/leukemia-gene-expression-cumida/Leukemia_GSE9476.csv'
    df = pd.read_csv(fn, index_col = 0)
    y = df['type']
    X = df.iloc[:,1:]


if dataset == 'Golub_1999_gen_expressions_for_leukemias':
    X,y = get_Golub_1999_gen_expressions_for_leukemias()

if dataset == 'GSE90047_Mouse_Liver_Hepatoblast_invivo':
    f = '/kaggle/input/trajectory-inference-single-cell-rna-seq/GSE90047_Mouse_Liver_Hepatoblast_invivo.csv'
    X = pd.read_csv(f ,  index_col= 0  )

if dataset == 'human_liver_ARCHS4':
    f = '/kaggle/input/human-liver-rnaseq-gene-expression-903-samples/human_liver.tsv'
    X = pd.read_csv(f ,  index_col= 0 , sep = '\t' )
    X = X.transpose()

if dataset == 'GSE67123_v6_scrna_10x_143_23548_Mouse_Embryo_HSCs_invivo_fromCytotrace':
    f = '/kaggle/input/genes-expressions-datasets-collection/GSE67123_v6_scrna_10x_143_23548_Mouse_Embryo_HSCs_invivo_fromCytotrace.csv'
    X = pd.read_csv(f ,  index_col= 0 )#  , sep = '\t' )
    
if dataset == 'single-cell-rna-seq-from-stoeckius-et-al-2017/GSE100866_CD8_merged-RNA_umi':
    f = '/kaggle/input/single-cell-rna-seq-from-stoeckius-et-al-2017/GSE100866_CD8_merged-RNA_umi.csv'
    X = pd.read_csv(f ,  index_col= 0 )#  , sep = '\t' )
    X = X.transpose()

if dataset == 'openml1146OVA_Prostate':
    dataset = openml.datasets.get_dataset(1146)
    X, y, categorical_indicator, attribute_names = dataset.get_data(
        dataset_format="dataframe", target=dataset.default_target_attribute )

if dataset == 'Dexter300from_scikit_hubness':
    f = '/kaggle/input/hubness-for-high-dimensional-datasets/y_Dexter300from_scikit_hubness.csv'
    y = pd.read_csv(f ,  index_col= 0  )
    y
    f = '/kaggle/input/hubness-for-high-dimensional-datasets/X_Dexter300from_scikit_hubness.csv'
    X = pd.read_csv(f ,  index_col= 0  )
    
    
X

In [ ]:
try:
    print(pd.Series(y).value_counts())
except:
    print('Seems y is not defined. There is no natural classes for the dataset')

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

import time

t0=time.time()
fig = plt.figure(figsize = (20,4) )

X = X.fillna(0)
X2 = PCA().fit_transform(X)
plt.scatter(X2[:,0],X2[:,1] ) #  , c = y  )
plt.title('PCA')
plt.grid()
plt.show()

print(time.time()-t0,'seconds passed')

In [ ]:

try:
    from sklearn.decomposition import PCA
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    pca = PCA(n_components=2)
    X = pca.fit_transform(X)

    plt.figure(figsize = (20,5))
    sns.scatterplot(X[:,0],X[:,1], hue = y) 
    
    plt.title('PCA')
    plt.grid()
    
except:
    print('Something goes wrong. Probably y is not defined')